In [1]:
import sys
sys.path.append('data_extraction/')

In [2]:
from reddit_data_extraction import *
from twitter_data_extraction import *


## Data

In [3]:
training_data_zip = ('data/rumoureval-2019-training-data.zip')
training_data_directory = ZipFile(training_data_zip)
training_data_contents = get_directory_structure(training_data_directory)

In [4]:
reddit_train_data = training_data_contents['reddit-training-data']
reddit_dev_data = training_data_contents['reddit-dev-data']

In [5]:
twitter_english = training_data_contents['twitter-english']

### Data Labels

In [6]:
train_labels = json.loads(training_data_directory.read('rumoureval-2019-training-data/train-key.json'))
dev_labels = json.loads(training_data_directory.read('rumoureval-2019-training-data/dev-key.json'))

### Reddit - Data Extraction

In [7]:
dev, train = get_train_dev(training_data_directory, reddit_train_data, reddit_dev_data, train_labels, dev_labels)

In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [9]:
dev_set = [d.pop('data', None) for d in dev]
train_set = [d.pop('data', None) for d in train]

In [10]:
development = pd.DataFrame(dev_set)
training = pd.DataFrame(train_set)
#training.head()

### Twitter - Data Extraction

In [11]:
data = get_twitter_data(training_data_directory, twitter_english, train_labels, dev_labels)
df = pd.DataFrame(data)
df[['text', 'depth', 'split', 'task_A_label']]

,text,depth,split,task_A_label
0,MT @euronews France: 10 dead after shooting at...,1,train,comment
1,@j0nathandavis They who? Stupid and partial op...,2,train,deny
2,"@nanoSpawn Socialists, Antisemites, anti zioni...",3,train,comment
3,@euronews @TradeDesk_Steve A French crime of p...,1,train,query
4,"@euronews LOL. 5 million Muslims in France, wh...",1,train,comment
...,...,...,...,...
5563,@emaccaz_ omfg it is 😱😱😱😱😱,2,train,comment
5564,@Angus_OL thank god they're all safe now. some...,1,train,support
5565,"@Angus_OL thank god its over, they're finally ...",1,train,comment
5566,@Angus_OL :(,1,train,comment


In [14]:
from embeddings import *

data_df = df.copy()

# Process Tweets
data_df = get_cleaned_tweets(data_df)

# Load W2V model
model = loadW2vModel()

# Create list of columns for each feature
# Create list of columns for each feature
vec_ID = list(range(1,301))
vec_labels = ['V{}'.format(i) for i in vec_ID]

vec_array = get_embeddings(data_df, model, vec_labels)

# add each element of the vector as an individual features to the dataframe
data_df = insert_embedding_feature(data_df, vec_array, vec_labels)


Loading the model
Done!


NameError: name 'vec_labels' is not defined